In [ ]:
import numpy as np
from tqdm import tqdm # 반복 연산에 대한 진행 상황을 추적

# 바이트를 벡터로 변환하는 임베딩 함수
def embed_bytes(byte):
    binary_string = f"{byte:08b}"
    vec = np.zeros(8)
    for i in range(8):
        if binary_string[i] == '1':
            vec[i] = float(1) / 16
        else:
            vec[i] = -float(1) / 16
    return vec

In [ ]:
import os

# 원시 PE 샘플의 위치를 읽어 레이블의 리스트를 만든다.
directories_with_lables = [("Benign PE Samples", 0), ("Malicious PE Samples", 1)]
list_of_samples = []
labels = []
for dataset_path, label in directories_with_lables:
    samples = [f for f in os.listdir(dataset_path)]
    for file in samples:
        file_path = os.path.join(dataset_path, file)
        list_of_samples.append(file_path)
        labels.append(label)
        
# 파일의 바이트 문자열을 읽을 편의 함수
def read_file(file_path):
    # 파일의 이진 문자열을 읽는다.
    with open(file_path, "rb") as binary_file:
        return binary_file.read()
    
# 샘플마다 읽을 바이트의 최대 길이(maxSize)를 설정
# 샘플의 모든 바이트를 임베딩하고, 결과를 X에 모은다.
max_size = 15000
num_samples = len(list_of_samples)
X = np.zeros((num_samples, 8, max_size))
Y = np.asarray(labels)
file_num = 0
for file in tqdm(list_of_samples):
    sample_byte_sequence = read_file(file)
    for i in range(min(max_size, len(sample_byte_sequence))):
        X[file_num, :, i] = embed_bytes(sample_byte_sequence[i])
    file_num += 1

In [ ]:
# Optimizer를 준비한다.
from keras import optimizers

my_opt = optimizers.SGD(lr=0.01, decay=1e-5, nesterov = True)

# Keras를 이용해 심층 신경망 구조를 설정한다.
from keras import Input
from keras.layers import Conv1D, Activation, multiply, GlobalMaxPool1D, Dense
from keras import Model

inputs = Input(shape=(8, max_size))
conv1 = Conv1D(kernel_size=128, filters=32, strides=128, padding='same')(inputs)
conv2 = Conv1D(kernel_size=128, filters=32, strides=128, padding='same')(inputs)
a = Activation('sigmoid', name='sigmoid')(conv2)
mul = multiply([conv1, a])
b = Activation('relu', name='relu')(mul)
p = GlobalMaxPool1D()(b)
d = Dense(16)(p)
predictions = Dense(1, activation='sigmoid')(d)
model = Model(inputs = inputs, outputs = predictions)

# 모델을 컴파일하고, 배치 크기를 선택함.
model.compile(optimizer=my_opt, loss="binary_crossentropy", metrics=['acc'])
batch_size = 16
num_batches = int(num_samples / batch_size)

# 배치에 대해 훈련을 진행한다.
for batch_num in tqdm(range(num_batches)):
    batch = X[batch_num * batch_size : (batch_num + 1) * batch_size]
    model.train_on_batch(batch, Y[batch_num * batch_size : (batch_num + 1) * batch_size])